In [2]:
# let's create a simple blockchain

In [6]:
import time
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import base64

In [56]:
class block:
    def __init__(self, prev_hash, data, nonce=None):
        # content of the block (e.g., transactions list)
        self.data = data
        
        # block header
        self.prev_hash = prev_hash
        self.timestamp = str(int(time.time()))
        if nonce:
            self.nonce = nonce
        else:
            self.nonce = base64.b16encode(os.urandom(16))
        
        # hash of the block header + data
        digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
        digest.update(self.prev_hash)
        digest.update(str.encode(self.timestamp))
        digest.update(self.nonce)
        digest.update(self.data)
        self.hash = base64.b16encode(digest.finalize())
    
    def find_hash(self):
        digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
        digest.update(self.prev_hash)
        digest.update(str.encode(self.timestamp))
        digest.update(self.nonce)
        digest.update(self.data)
        return base64.b16encode(digest.finalize())
    
    
    def find_nonce(self, difficulty):
        prefix = b'0' * difficulty
        while True:
            self.nonce = os.urandom(16)
            msg_digest = self.find_hash()
            if msg_digest.startswith(prefix):
                self.hash = msg_digest
                break
        
        
    def __repr__(self):
        return 'PreviousHash: {}\nTimestamp: {}\nNonce: {}\nHash {}'.format(
            self.prev_hash, self.timestamp, self.nonce, self.hash)
        

In [11]:
test_block = block(b'1', b'dummy data')

In [12]:
test_block

PreviousHash: b'1'
Timestamp: 1528230063
Nonce: b'2E90569155D20D64330267EDF5C3A24C'
Hash b'278C3530267AFF39E3511816A13F5F1AFD134AE8B34039BB31AADCB919E928DE'

In [14]:
genesis_block = block(
b'0000000000000000000000000000000000000000000000000000000000000000',
b'PyCon 2018 Genesis Block',
b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF')

In [15]:
genesis_block

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528231669
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'

In [48]:
class blockchain:
    def __init__(self):
        self.blocks = []
        
    def add_block(self, block):
        self.blocks.append(block)
        
    def blockchain_valid(self):
        if self.blocks[0] != genesis_block:
            return False
        prev_block = self.blocks[0]
        
        for block in self.blocks[1:]:
            # Check if the prv_hash of the block points to the prv block
            if prev_block.hash != block.prev_hash:
                return False
            
            # Check Hash(current+prv) = current hash
            digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
            digest.update(block.prev_hash)
            digest.update(str.encode(block.timestamp))
            digest.update(block.nonce)
            digest.update(block.data)
            hash_digest = base64.b16encode(digest.finalize())
            
            if hash_digest != block.hash:
                return False
            
            prev_block = block

        return True
    
    def __repr__(self):
        _blocks = []
        for _block in self.blocks:
            _blocks.append(str(_block))
        return '\n\n'.join(_blocks)

In [32]:
test_blockchain = blockchain()

In [33]:
test_blockchain

In [34]:
test_blockchain.add_block(genesis_block)

In [35]:
test_blockchain.blocks

[PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
 Timestamp: 1528231669
 Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
 Hash b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295']

In [36]:
test_blockchain

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528231669
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'

In [37]:
print(test_blockchain.blockchain_valid())

True


In [38]:
# bad genesis block
test_blockchain = blockchain()
test_blockchain.add_block(block(b'1', b'2'))
print(test_blockchain.blockchain_valid())

False


In [39]:
# corect blocks
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)
print(test_blockchain.blockchain_valid())

print(genesis_block)
test_block = block(genesis_block.hash, b'dummy data')
print(test_block)
test_blockchain.add_block(test_block)
test_blockchain.blockchain_valid()

True
PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528231669
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'
PreviousHash: b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'
Timestamp: 1528233075
Nonce: b'7C74029273AFDAFD5A0A5BA70B7CEE8B'
Hash b'A188886C718B6AC41A58D1E87CDF13A0B165DE2070AC08596D73F320B3B2989F'


True

In [41]:
test_block2 = block(test_block.hash, b'dummy data2')
print(test_block2)
test_blockchain.add_block(test_block2)
test_blockchain.blockchain_valid()

PreviousHash: b'A188886C718B6AC41A58D1E87CDF13A0B165DE2070AC08596D73F320B3B2989F'
Timestamp: 1528290108
Nonce: b'7B7F36E82051B90315BA9E2982C73D14'
Hash b'C84889DEDDAC1B578EE4F4EF8B77C88402974784C0C4C3579D0206C35309F70D'


True

In [42]:
test_blockchain

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528231669
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'

PreviousHash: b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'
Timestamp: 1528233075
Nonce: b'7C74029273AFDAFD5A0A5BA70B7CEE8B'
Hash b'A188886C718B6AC41A58D1E87CDF13A0B165DE2070AC08596D73F320B3B2989F'

PreviousHash: b'A188886C718B6AC41A58D1E87CDF13A0B165DE2070AC08596D73F320B3B2989F'
Timestamp: 1528290108
Nonce: b'7B7F36E82051B90315BA9E2982C73D14'
Hash b'C84889DEDDAC1B578EE4F4EF8B77C88402974784C0C4C3579D0206C35309F70D'

In [ ]:
# proof of work

In [57]:
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)

In [58]:
test_blockchain

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528231669
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'

In [59]:
test_block = block(genesis_block.hash, b'dd')
test_block

PreviousHash: b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'
Timestamp: 1528291659
Nonce: b'3DC4E50F09CD889227CF35A646A27C6C'
Hash b'A6B1DC797223A1946008A2FDAA7ADA6C709E0B952CF6C1CE329FC9F7A5703058'

In [60]:
test_block.find_nonce(3)
test_block

PreviousHash: b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'
Timestamp: 1528291659
Nonce: b'ch\xc8j\x1f2\xfa\x1c\x8f\xe3\xbd/\xd9\xa3a\xd1'
Hash b'0000301A8ADFCF9C65F35823116B392736AB60CC84A882F6375CA89FFB287E88'

In [61]:
# tale of two blockchains

In [62]:
test_blockchain1 = blockchain()
test_blockchain2 = blockchain()

test_blockchain1.add_block(genesis_block)
test_blockchain2.add_block(genesis_block)

print(test_blockchain1)
print()
print(test_blockchain1)

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528231669
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528231669
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'1AD9C383EA0A0B80DA7EA90072DCEE80025A68EFAE263E5F24EC0CAF65950295'


In [ ]:
def mine_blocks(test_blockchain):
    s_time = int(time.time())
    duration = 10 
    
    _prv_block = genesis_block
    
    while True:
        _block = block(_prv)
    